# HW1 - Intro to the Map Reduce Paradigm  
__ `MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Spring 2018`__

Welcome to Machine Learning at Scale! This first homework assignment introduces one of the core strategies in distributed processing: divide and conquer. We'll use the simplest of tasks, word counting, to illustrate the difference between a scalable and non-scalable algorithm. You will be working with the text of _Alice in Wonderland_ to put these ideas into practice using Python and Bash scripting. By the end of this week you should be able to:
* ... __describe__ the Bias-Variance tradeoff as it applies to Machine Learning.
* ... __explain__ why we consider word counting to be an "Embarrassingly Parallel" task.
* ... __estimate__ the runtime of embarrassingly parallel tasks using "back of the envelope" calculations.
* ... __implement__ a Map Reduce algorithm using the Command Line.
* ... __set-up__ a Docker container and know why we use them for this course.

You will also  become familiar (if you aren't already) with `defaultdict`, `re` and `time` in Python, linux piping and sorting, and Jupyter magic commands `%%writefile` and `%%timeit`. 


__IMPORTANT:__ If you're not familiar with linux, you should read the following tutorial reagrding **piping** and **redirecting**: https://ryanstutorials.net/linuxtutorial/piping.php You will need to understand the differences to answer some of the later questions.

__Please refer to the `README` in github for detailed submission instructions__.


# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [1]:
# confirm you are running Python 3
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=5, releaselevel='final', serial=0)

In [2]:
# imports
import re
import sys

Create a folder for any data you download locally.

In [3]:
!mkdir data
# NOTE: the contents of this directory will be ignored by git.

mkdir: cannot create directory `data': File exists


# Question 1: Introductions

`The Caterpillar and Alice looked at each other for some time in silence: at last the Caterpillar took the hookah out of its mouth, and addressed her in a languid, sleepy voice. "Who are you?" said the Caterpillar.`   
<div style="text-align: right"> -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4 </div>


__Tell us about yourself! Briefly describe where you live, how far along you are in MIDS, what other classes you are taking and what you want to get out of w261.__

> Type your response here!

# Question 2: Bias - Variance
__In 1-2 sentences explain the bias-variance trade off. Describe what it means to "decompose" sources of error. How is this used in machine learning?__ Please use mathematical equation(s) to support your explanation. (Use `$` signs to take advantage of $\LaTeX$ formatting, eg. `$f(x)$` will look like: $f(x)$). Please also cite any sources that informed your answer.

> Type your response here!

### --- SOLUTION ---
> The bias–variance tradeoff is a central problem in supervised learning. Ideally, one wants to choose a model that both accurately captures the regularities in its training data, but also generalizes well to unseen data. Unfortunately, it is typically impossible to do both simultaneously. High-variance learning methods may be able to represent their training set well, but are at risk of overfitting to noisy or unrepresentative training data. In contrast, algorithms with high bias typically produce simpler models that don't tend to overfit, but may underfit their training data, failing to capture important regularities.

> Models with low bias are usually more complex (e.g. higher-order regression polynomials), enabling them to represent the training set more accurately. In the process, however, they may also represent a large noise component in the training set, making their predictions less accurate - despite their added complexity. In contrast, models with higher bias tend to be relatively simple (low-order or even linear regression polynomials), but may produce lower variance predictions when applied beyond the training set.

> In the case of regression problems where we estimate mean squared error (MSE), we can decompose the overall expected error in terms of the squared bias, variance, and irreducible error:  


$$E[(\hat{y}-y)^2]=(E[\hat{y}]-y_{true})^2+E[\hat{y}-E[\hat{y}]]^2+E[(y_{true}-y)^2]$$

$$E[(y-h(x))^2]  = Bias[h(x)]^2 + Variance[h(x)] + +E[(y_{true}-y)^2]$$

> Where the h(x) is the model built from each boostrap sample of the test/heldout dataset (see details below).

 
> The first term on the RHS is the squared bias, which measures the average error of the model.  The second term is the variance, which measures how much our model's predictions vary from one training set to another.  The final term represent the irreducible error; the variation between our model and reality that we are unable to do anything about (thus the name "irreducible error").

> Estimating bias and variance for polynomial models on a dataset with unknown underlying functional form will generally be predicated on a bootstrapping procedure, creating an ensemble of models for each polynomial degree. The model variance will be calculated as the average deviation from the average model (for each degree). Measuring bias is unfortunately somewhat less direct, as one cannot separate the irreducible error (noise) present in the data. To determine an estimate of model bias, one can compute the average squared error (across an ensemble) from the testing data and subtract off the model variance, rendering a value that is the sum of irreducible error and squared bias. Ultimately, to choose a model degree one can minimize the overall error, but should also be sure to keep a balance between bias and variance indivdiually, aiming to keep both as low as possible.


# Question 3: Tokenizing
A number of our assignments this term will involve extracting information from text. A common preprocessing step when working with raw files is to 'tokenize' (i.e. extract words from) the text. Within the field of Natural Language Processing a lot of thought goes into what specific tokenizing makes most sense for a given task. For example, you might choose to remove punctuation or to consider punctuation symbols  'tokens' in their own right. __In this question you'll use the Python `re` module to create a tokenizer to use when you perform WordCount on the _Alice In Wonderland_ text.__

### Q3 Tasks:

* __a) short response:__ In the Naive Bayes algorithm (which we'll implement next week), we'll estimate the _likelihood_ of a word by counting the number of times it appears and dividing by the size of the vocabulary (total number of unique words). Using the text: *"Alice had an adventure that took alice to wonderland"*, give a concrete example of how two different tokenizers could cause us to get two different results on this calculation. [`HINT`: _you should not need to read up on Naive Bayes to answer this question_]  
  

* __b) short response:__ When tokenizing in this assignment we'll remove punctuation and discard numerical digits by making everything lowercase and then capturing only consecutive letters a to z. Suppose __`tokenize(x)`__ is a Python function that performs the desired tokenization. What would __`tokenize("By-the-bye, what became of Alice's 12 hats?!")`__ output? Type the answer in the space provided below.


* __c) code:__  Fill in the regular expression pattern in the cell labeled `part c` so that the subsequent call to `re.findall(RE_PATTERN, ...)` returns the tokenization described above. [`HINT`: _we've taken care of the lowercase part for you. If regex is new to you, check out the [`re`  documentation](https://docs.python.org/3/library/re.html) or [this PyMOTW tutorial](https://pymotw.com/2/re/)._]

### Q3 Student Answers:
> __a)__ Type your answer here!

> __b)__ Type your answer here!

### --- SOLUTION ---
> __a)__ Different tokenizers might cause certain raw words to get counted as the same or as distinct tokens which would affect both the total number of (unique) words and the individual word counts for those words. For example, suppose we had the following small corpus: `"Alice had an adventure that took alice to wonderland"`... a tokenizer that doesn't lowercase words would calculate the likelihood of 'alice' to be    $\frac{1}{9}$ whereas if our tokenizer did lowercase all words the likelihood of 'alice' would be $\frac{2}{8}$

> __b)__ `['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']`

In [4]:
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("")          

In [5]:
# < --- SOLUTION --- >
# Part C - Fill in the regular expression
RE_PATTERN = re.compile("[a-zA-Z]+") 

In [6]:
# Tokenize - DO NOT MODIFY THIS CELL, just run it as is to check your pattern
words = re.findall(RE_PATTERN, "By-the-bye, what became of Alice's 12 hats?!".lower())
print(words)

['by', 'the', 'bye', 'what', 'became', 'of', 'alice', 's', 'hats']


# Load the Data
` "Please would you tell me", said Alice, a little timidly, for she was not quite sure whether it was good manners for her to speak first, "why your cat grins like that?"`  
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 4  </div>

For the main part of this assignment we'll be working with the free plain text version of _Alice's Adventures in Wonderland_ available from Project Gutenberg. __Use the first two cells below to download this text and preview the first few lines.__ 

In [7]:
# Download Full text 
# NOTE: feel free to replace 'curl' with 'wget' or equivalent command of your choice.
!curl "http://www.gutenberg.org/files/11/11-0.txt" -o data/alice.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  170k  100  170k    0     0   571k      0 --:--:-- --:--:-- --:--:--  571k


In [8]:
# Take a peak at the first few lines
!head -n 6 data/alice.txt
# NOTE: If you are working in JupyterLab on Docker you may not see the output 
# below due to an encoding issue... in that case, use a terminal on Docker to 
# execute this head command and confirm that the file has downloaded properly, 
# this encoding issue should not affect your work on subsequent HW items.

﻿The Project Gutenberg EBook of Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


We'd also like you to develop a habit of creating small files with simulated data for use in developing, debugging and testing your code. The jupyter magic command `%%writefile` is a convenient way to do this. __Run the following cells to create a test data file for use in our word counting task.__

In [9]:
%%writefile data/alice_test.txt
This is a small test file. This file is for a test.
This small test file has two small lines.

Overwriting data/alice_test.txt


In [10]:
# confirm the file was created in the data directory using a grep command:
!ls data | grep test

alice_test.txt
alice_test.txt.output


# Question 4: Word Count in Python

Over the course of this term you will also become very familiar writing Python programs that read from standard input and using Linux piping commands to run these programs and save their output to file. __In this question you will write a short python script to perform the Word Count task and then run your script on the _Alice in Wonderland_ text__. You can think of this like a 'baseline implementation' that we'll later compare to the parallelized version of the same task.

### Q4 Tasks:

* __a) code:__ Complete the Python script in the file __`wordCount.py`__. Read the docstrings carefully to be sure you understand the expected behavior of this function. Please do not code outside of the marked location.


* __b) testing:__ Run the cell marked `part b` to call your script on the test file we created above. Confirm that your script returns the correct counts for each word by visually comparing the output to the test file. 


* __c) results:__ When you are confident in your implementation, run the cell marked `part c` to count the number of occurrences of each word in _Alice's Adventures in Wonderland_. In the same cell we'll pipe the output to file. Then use the provided `grep` commands to check your answers.


* __d) short response:__ Suppose you decide that you'd really like  a word and its plural (e.g. 'hatter' and 'hatters' or 'person' and 'people') to be counted as the same word. After we have run the wordcount would it be more efficient to post-process your output file or discard your output file and start the analysis over with a new tokenizer? Explain your reasoning briefly. 

### Q4 Student Answers:
> __a-c)__ _Complete the coding portions of this question before answering 'e'._

> __d)__ Type your answer here!

### --- SOLUTION ---
> __d)__ Assuming that the vocabulary list is much shorter than the length of the original document it would be more efficient to postprocess our counts and combine the necessary records because this would involve looking at fewer records.

In [11]:
# part a - DO YOUR WORK IN wordCount.py

In [12]:
# part b - DO NOT MODIFY THIS CELL, just run it as is to test your script
!python wordCount.py < data/alice_test.txt

this	3
is	2
a	2
small	3
test	3
file	3
for	1
has	1
two	1
lines	1


In [13]:
# part c - DO NOT MODIFY THIS CELL, just run it as is to perform the word count.
!python wordCount.py < data/alice.txt > data/alice_counts.txt

Take a look at the first 10 words & their counts.

In [14]:
!head data/alice_counts.txt

the	1832
project	88
gutenberg	98
ebook	11
of	635
alice	404
s	224
adventures	12
in	435
wonderland	8


__Check your results:__ How many times does the word "alice" appear in the book? 

In [15]:
# EXPECTED OUTPUT: 404
!grep alice data/alice_counts.txt

alice	404


__Check your results:__ How many times does the word "hatter" appear in the book? 

In [16]:
# EXPECTED OUTPUT: 56
!grep hatter data/alice_counts.txt

hatter	56
hatters	1


__Check your results:__ How many times does the word "queen" appear in the book? 

In [17]:
# EXPECTED OUTPUT: 76
!grep queen data/alice_counts.txt

queen	76
queens	1


# Question 5: Unix Sorting
Another common task in this course's assignments will be to make strategic use of sorting. To illustrate what we mean by 'strategic,' let's return to the (admitedly artificial) scenario posed in Question 4d): _we have a file of word counts but we want to find all pairs of words and plurals (eg. 'queen' and 'queens') to join their counts into a single record_. Suppose we were going to do this by hand. A logical approach would be to start with the first word and read through each subsequent word to check if they are a singular-plural pair, then repeat this process for each word in the file.    

__IMPORTANT NOTE__: For this question, pretend that all plurals simply end with 's'.

### Q5 Tasks:
* __a) short response + code:__ In the worst case scenario, how many comparisons will we have to make to perform the process described above? Use $\LaTeX$ formatting to show the calculations that led to your answer. What is the Big O complexity of this "algorithm"? [`HINTs:` _To answer the first part of this question you'll want to know how many words are in your_ `alice_counts.txt` _file -- in the space provided below, write a unix command to check. If you need a Big O notation refresher, here's_ a [blog post](https://rob-bell.net/2009/06/a-beginners-guide-to-big-o-notation/), a [cheatsheet](http://bigocheatsheet.com), and a [thorough explanation](http://pages.cs.wisc.edu/~vernon/cs367/notes/3.COMPLEXITY.html).]


* __b) short response:__ If the word count file were sorted alphabetically, how would your approach to this task change? How many comparisons would you have to make? What would the Big O complexity of this new strategy be? *(Remember that for this question, we're making the simplifying assumption that all plural words simply end with 's')*


* __c) short response:__ What is the Big O complexity of the fastest sorting algorithms? When we take the time to sort into account, does it make sense to alphabetize before combining singular-plural pairs? [`HINT:` *When it comes to sorting algorithms -- Wikipedia is your friend, as is the cheatsheet from part a.*]


* __d) code:__ Write a unix command to sort your word count file alphabetically. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) the results to `data/alice_counts_A-Z.txt`. [`HINT:` if Unix sort commands are new to you, start with [this biowize blogpost](https://biowize.wordpress.com/2015/03/13/unix-sort-sorting-with-both-numeric-and-non-numeric-keys/) or [this unixschool tutorial](http://www.theunixschool.com/2012/08/linux-sort-command-examples.html)]


* __e) code:__ A (slightly) more likely scenario is that we'd want to look at the top 10 most frequent words that appear in the book. Write a unix command to sort your word count file from highest to lowest count. Save (i.e. [redirect](https://superuser.com/questions/277324/pipes-vs-redirects)) your results to `data/alice_counts_sorted.txt`; then run the provided cell to print the top ten words. Compare your output to the expected output we provide.

### Q5 Student Answers:
> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!


### --- SOLUTION ---
> __a)__ In the worst case scenario each word would have to be compared to all subsequent words leading to   $3019 + 3018 + ... 1 = \frac{1}{2} * 3019(3019 - 1) = 4,555,671$ comparisons. This 'algorithm' is $O(n^2)$. 

> __b)__ If instead our file were sorted alphabetically, we would only have to compare adjacent words... i.e. $3019$ comparisons, $O(n)$.

> __c)__ The worst case scenario time complexity of the fastest sorting algorithms (mergesort/quicksort/heapsort) are $O(n \cdot logn)$. If we add this computation time to the time of our more efficient combining strategy from 'b' we get a total time complexity of $O(n) + O(n\cdot logn) \approx O(n \cdot logn)$ which is still faster than $O(n^2)$ so yes, it makes sense to alphabetize in this case. 


In [18]:
# part a - write a unix command to check how many records are in your word count file


In [19]:
# < --- SOLUTION --- >
# part a - write a unix command to check how many records are in your word count file
!wc -l data/alice_counts.txt                        

3019 data/alice_counts.txt


In [20]:
# part d - unix command to sort your word counts alphabetically 


In [21]:
# < --- SOLUTION --- >
# part d - unix command to sort your word counts alphabetically
!sort data/alice_counts.txt > data/alice_counts_A-Z.txt  

In [22]:
# part d - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_A-Z.txt   

a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2


In [23]:
# part e - unix command to sort your word counts from highest to lowest count


In [24]:
# < --- SOLUTION --- >
# part e - unix command to sort your word counts from highest to lowest count
!sort -k2,2nr data/alice_counts.txt > data/alice_counts_sorted.txt  

In [25]:
# part e - DO NOT MODIFY THIS CELL, run it as is to confirm your sort worked
!head data/alice_counts_sorted.txt  

the	1832
and	945
to	810
a	695
of	635
it	610
she	553
i	546
you	484
said	462


<table>
<th>expected output for (d):</th>
<th>expected output for (e):</th>
<tr><td><pre>
a	695
abide	2
able	1
about	102
above	3
absence	1
absurd	2
accept	1
acceptance	1
accepted	2
</pre></td>
<td><pre>
the	1832
and	945
to	810
a	695
of	635
it	610
she	553
i	546
you	484
said	462
</pre></td></tr>
</table>

# Question 6: Parallel Word Count (part 1)
What would happen if we tried to run our script on a much larger dataset? For one thing, it would take longer to run. However there is a second concern. The Python object that aggregates our counts (`defaultdict`) exists in memory on the machine running this notebook. If the vocabulary is too large for the memory space available we would crash the notebook. The solution? Divide and Conquer! Instead of running the script on the whole dataset at once, we could split our text up in to smaller 'chunks' and process them independently of each other. __In this question you'll use a bash script to "parallelize" your Word Count.__


### Q6 Tasks:
* __a) read provided code:__ The bash script `pWordCount_v1.sh` takes an input file, splits it into a specified number of 'chunks', and then applies an executable of your choice to each chunk. Read through this code and make sure you understand each line before you proceed. [`HINT:` _For now, ignore the 'student code' section -- you'll use that in part c._]


* __b) short response:__ Below we've provided the command to use this script to apply your analysis (`wordCount.py`) to the _Alice_ text in 4 parallel processes. We'll redirect the results into a file called `alice_pCounts.txt.` Run this analysis and compare the count for the word 'alice' to your answer from Question 4. Explain what went wrong and describe what we have to add to `pWordCount_v1.sh` to fix the problem.


* __c) code:__ We've provided a python script, `aggregateCounts_v1.py`, which reads word counts from standard input and combines any duplicates it encounters. Read through this script to be sure you understand how it is written. Then follow the instructions in `pWordCount_v1.sh` to make a one-line modification so that it accepts `aggregateCounts_v1.py` as a 4th argument and uses this script to combine the chunk-ed word counts. Run the cell below to confirm that you now get the correct results for your 'alice' count.

### Q6 Student Answers:
> __b)__ Type your answer here!

### --- SOLUTION ---
> __b)__ Instead of a single count our file includes four separate counts for the word 'alice.' This is because each of our data chunks were counted separately by a separate mapper. To solve this problem we'd need to add another phase to our job in which we combine the results of each mapper into a single count for each word.

In [24]:
# part b - make sure your scripts are executable (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v1.sh
!chmod a+x wordCount.py

In [25]:
# part b - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' 'wordCount.py' > 'data/alice_pCounts.txt'

In [26]:
# part b - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	113
alice	126
alice	122
alice	43


In [29]:
# part c - make sure the aggregateCounts script is executable (RUN THIS CELL AS IS)
!chmod a+x aggregateCounts_v1.py

In [30]:
# part c - parallel word count on Alice text (RUN THIS CELL AS IS)
!./pWordCount_v1.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v1.py' \
                   > 'data/alice_pCounts.txt'

In [31]:
# part c - check alice count (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	404


In [32]:
!man sort

No manual entry for sort


# Question 7: Parallel Word Count (part 2)
Congratulations, you've just implemented a Map-Reduce algorithm! From here on out, we'll refer to the two python scripts you passed to `pWordCount_v1.sh` as '_mapper_' and '_reducer_'. The bash script itself served as our '_framework_' -- it split up the original input, then ___mapped___ our word counting script on to each chunk, then ___aggregated (a.k.a. reduced)___ the resulting files by piping them into our collation script.  Unfortunately, as you may have realized already, there is a major scalability concern with this particular implementation. __In this question you'll fix our implementation of parallel word count so that it will be scalable.__

__HINT:__ MapReduce uses the Merge-Sort algorithm under the hood. Linux `sort` command has a merge option which you can use to simiulate the MapReduce framework. Use the `man sort` command to find more information on this option. 

### Q7 Tasks:

* __a) short response:__ What is the potential scalability problem with the provided implementation of `aggregateCounts_v1.py`? Explain why this supposedly 'parallelized' Word Count wouldn't work on a really large input corpus.  [`HINT:` _see the intro to Q6_]


* __b) code:__ Fortunately, a 'strategic sort' can solve this problem. Read the instructions at the top of `pWordCount_v2.sh` carefully then make your changes that alphabetically sort the output from the mappers (`countfiles`) before piping them into the reducer script.


* __c) code:__ Write the main part of `aggregateCounts_v2.py` so that it takes advantage of the sorted input to add duplicate counts without storing the whole vocabulary in memory. Refer to the file docstring for more detailed instructions. Confirm that your implementation works by running it on both the test and true data files.


* __d) short response:__ If you are paying close attention, this rewritten reducer sets us up for a truly scalable solution, but doesn't get us all the way there. In particular, while we chunked our data so it can be processed by multiple mappers, we're still streaming the entire dataset through one reduce script. If the vocabulary is too large to fit on a single computer, we might split the word counts in half after sorting them, then perform the reducing on two separate machines. Explain what could go wrong with this approach. (For now, ignore the question of how we'd sort a dataset that is too large to fit on a single machine and just focus on what might be wrong about the result of this split-in-half reducing).


* __e) short response:__ Can you come up with a different way of splitting up the data that would allow us to perform the reducing on separate machines without needing any postprocessing? This is a theoretical question -- don't worry if you don't know how to implement your idea in a bash script, just describe how you'd want to split the sorted counts into different files to be reduced separately.

### Q7 Student Answers:

> __a)__ Type your answer here!

> __b-c)__ _complete the coding portions of this question before answering d and e._

> __d)__ Type your answer here!

> __e)__ Type your answer here!

### --- SOLUTION ---
> __a)__ The provided implementation in `aggregateCounts_v1.py` stores the entire vocabulary in a dictionary while accumulating the counts. For a small corpus this is fine, but for a really large corpus and/or a cluster of machines with memory constraints this dictionary could become too large to run on a single node. In fact, from a scalability perspective this implementation is essentially equivalent to our original python word counter.

> __d)__ If we were to split the sorted mapper output exactly in half there is a chance that the middle word in the vocabulary would have some of its sub-counts sent to the first reducer and others sent to the second reducer... as a result our final file would include a duplicate key with un-aggregated counts.

> __e)__ We could avoid this potential problem if we could group records by word so that all the records from any given word end up in the same partition. For example this would happen if we partition based on the first letter in the word.

In [ ]:
# Add permissions to your new files (RUN THIS CELL AS IS)
!chmod a+x pWordCount_v2.sh
!chmod a+x aggragateCounts_v2.sh

In [33]:
# part c - test your code on the test file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice_test.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py'

a	2
file	3
for	1
has	1
is	2
lines	1
small	3
test	3
this	3
two	1


In [34]:
# part c - run your code on the Alice file (RUN THIS CELL AS IS)
!./pWordCount_v2.sh 4 'data/alice.txt' \
                   'wordCount.py' \
                   'aggregateCounts_v2.py' \
                   > 'data/alice_pCounts.txt'

In [35]:
# part c - confirm that your 'alice' count is correct (RUN THIS CELL AS IS)
!grep alice data/alice_pCounts.txt

alice	404


# Question 8: Scalability Considerations
In your reading for Week 2's live session, [Chapter1, section 2](https://lintool.github.io/MapReduceAlgorithms/MapReduce-book-final.pdf) of _Data Intensive Text Processing with MapReduce_, Lin and Dyer discuss a number of "Big Ideas" that underlie large scale processing: __scale "out," not "up"; assume failures are common; move processing to the data; process data sequentially and avoid random access; hide system-level details from the application developer; and seamless scalability.__ Part of our work this semester will be to interact with these ideas in a practical way, not just a conceptual one.

### Q8 Tasks:
* __a) short response:__ What do Lin and Dyer consider the two features of an "ideal algorithm" from a scalability perspective?


* __b) short response:__ The mapper script below (created on the fly using a little Jupyter magic) will help us illustrate the concept of scalability. Run the provided code which passes this mapper script to our parallel computation 'framework' and runs the 'analysis' on the _Alice_ text file. Note that we've omitted a reducer for simplicity. What do you observe about the time it takes to run this "algorithm" when we use 1, 2 and 4 partitions? Does it meet Lin and Dyer's criteria?


* __c) short response:__ Let's try something similar with your Word Count analysis. Run the provided code to time your implementation with 1, 2, 4 and 8 partitions. What do you observe about the runtimes? Does this match your expectation? Speculate about why we might be seeing these times. What conclusions should we draw about the scalability of our implementation? [`HINT:` _consider the limitations of both your machine and our implementation... there are some competing forces at work, what are they?_]


* __d) OPTIONAL:__ Which components of your Map-Reduce algorithm are affected by a change in the number of partitions? Does increasing the number of partitions increase or decrease the total time spent on each of these portions of the task? What tradeoff does this cause?

### Q8 Student Answers:

> __a)__ Type your answer here!

> __b)__ Type your answer here!

> __c)__ Type your answer here!

> __d)__ Type your (OPTIONAL) answer here!

### --- SOLUTION ---
> __a)__ Ideally an algorithm will process twice the data in no more than twice the time and given the same size data, doubling the number of parallel processes should at least half the runtime.

> __b)__ Yes, doubling the number of map tasks (approximately) halves the runtime.

> __c)__ When we time our word count algorithm we don't see the perfect scalability that we saw with the demo mapper. Specifically the times are close to equal for 1,2 and 4 partitions but roughly double as we increase to 8 then 16 then 32. One reason the times are similar for 1, 2 and 4 is that while increasing the number of partitions makes the map phase quicker this comes at the expense of an increase in the amount of sorting that has to happen (and consequently the time to sort). The likely reason for the much longer times for 8, 16 and partitions is that I am running on a machine that has only dedicated 4 cores to Docker so I can only parallelize 4 processes simultaneously, when I specify more than 4 mappers some mappers end up waiting for others to finish before they can start. We should not conclude that our algorithm isn't scalable -- it's just not 'ideally' scalable. [__Important note:__ _your times may vary depending on your machine, your discussion should match your results which might be different that those presented here._] 

> __d)__ As noted above... the map/reduce phases will be shorter with more partitions but the sort will take longer because there will be more records to shuffle. 

__Run the following cells to create the mapper referenced in `part b`__

In [45]:
!mkdir demo

In [50]:
%%writefile demo/mapper.py
#!/usr/bin/env python
"""
This mapper reads from STDIN and waits 0.001 seconds per line.
Its only purpose is to demonstrate one of the scalability ideas.
"""
import sys
import time
for line in sys.stdin:
    time.sleep(0.001)

Overwriting demo/mapper.py


In [51]:
# Make sure the mapper is executable
!chmod a+x demo/mapper.py

__Run the next three cells to apply our demo mapper with 1, 2 and 4 partitions.__

In [52]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'demo/mapper.py'

5.26 s ± 57.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'demo/mapper.py'

2.75 s ± 22.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'demo/mapper.py'

1.45 s ± 30.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


__Run the following cells to repeat this process with your word count algorithm.__

In [55]:
%%timeit
!./pWordCount_v2.sh 1 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

256 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
%%timeit
!./pWordCount_v2.sh 2 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

249 ms ± 5.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%%timeit
!./pWordCount_v2.sh 4 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

254 ms ± 7.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%%timeit
!./pWordCount_v2.sh 8 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

273 ms ± 4.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%%timeit
!./pWordCount_v2.sh 16 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

382 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%%timeit
!./pWordCount_v2.sh 32 'data/alice.txt' 'wordCount.py' 'aggregateCounts_v2.py' > 'data/tmp'

535 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Question 9: Embarrassingly Parallel
`"If any one of them can explain it," said Alice, (she had grown so large in the last few minutes that she wasn’t a bit afraid of interrupting him,) "I’ll give him sixpence. I don’t believe there’s an atom of meaning in it."`
<div style="text-align: right">  -- Lewis Carroll, _Alice's Adventures in Wonderland_, Chapter 12  </div>

### Q9 Tasks:

* __a) short response:__ Describe what we mean by 'Embarrassingly Parallel' in terms of word counting. Does this term describe a 'task'? an 'implementation of a task'? 

* __b) OPTIONAL__: Define this concept in terms of 'associative' and 'commutative' operations. [`HINT:` _Refer to Chapter 2 in DITP_ ]

### Q9 Student Answers:

> __a)__ Type your answer here!

> __b)__ Type your (OPTIONAL) answer here!

### --- SOLUTION ---
> "Embarassingly Parallel" describes the category of parallel computation tasks that can be implemented in a way that the subtasks rarely or never have to communicate. More specifically we can think of this as the kind of algorithm that can be performed completely in parallel except, possibly, for a single join at the end. Word counting fits this description since we can split the input file into as many subfiles as we like and perform sub counts then join once at the end. More generally any calculation that can be defined in terms of commutative and associative operations on the input records will be embarassingly parallel.    
(Async 2.3)

> b) The word "associative" comes from "associate" or "group"; the Associative Property is the rule that refers to grouping.In the "Embarrassingly Parallel" word count problem, you can group the workload into multiple subtasks in any way you want, summing the results at the ends will give you the same result.
The word "commutative" comes from "commute" or "move around".In the same word count problem, you can sum up the counts in any order you like. Hence, "Embarrassingly parallel" problems exhibits both associative and commutative properties


>mappers need be associative -> (a)+(b+c) = (a+b)+(c) chunks can be split any way and processed independently   
reducers need be commutative -> (a)+(b+c) = (b+c)+(a) chunks can be combined in any order   
(Async 3.6)

### Congratulations, you have completed HW1! Please refer to the readme for submission instructions.

If you would like to provide feedback regarding this homework, please use the survey at: https://docs.google.com/forms/d/e/1FAIpQLSce9feiQeSkdP43A0ZYui1tMGIBfLfzb0rmgToQeZD9bXXX8Q/viewform